#### FS20P2 - Projekt Climate Change 
Team: Firat Saritas & Kajenthini Kobivasan

# Abgabe für Grundkompetenz Datenvisualisierung, Interaktive Datenvisualisierung & Digital kommunizieren

### Import library

In [1]:
import dash
import dash_html_components as html
import dash_core_components as dcc
import dash_table
import pandas as pd
from dash.dependencies import Input, Output, State
import requests
from bs4 import BeautifulSoup as bs
import re
import numpy as np
import itertools
import mysql.connector
import mysql
import pymysql
import paramiko
from paramiko import SSHClient
from sshtunnel import SSHTunnelForwarder
from os.path import expanduser
from io import StringIO
import matplotlib.pyplot as plt
import base64
import plotly
import plotly.express as px

### Python-Verbindung mit MySQL-Server herstellen:

Wir haben einen Switch Server von der FHNW beantragt und dort eine Ubunutu-Umgebung eingerichtet.
Auf diesem Server haben wir einen MySQL Datenbank erstellt.
Die Zugangsdaten zur Datenbank sieht wie folgt aus:

In [2]:
sql_hostname = '127.0.0.1'
sql_username = 'root'
sql_password = 'Alstom2019ge'
sql_main_database = 'climate'
sql_port = 3306
ssh_host = '86.119.42.13'
ssh_user = 'ubuntu'
ssh_port = 22
sql_ip = '1.1.1.1.1'
shh_key = 'key.txt'

### Die Daten

Durch Web Scraping haben wir unsere globalen und lokalen Daten erhoben. Die Codes sind so geschrieben, sodass falls die Webseite oder die CSV-Datei erneuert wird, die alte Tabelle in der Datenbank gelöscht und eine neue Tabelle mit den aktualisierten Daten erstellt wird.
Falls sich fehlenden Angaben (na) in den Tabellen auf der Webseite oder in den CSV-Dateien sich befinden, wird direkt die ganze Zeile entfernt.

### Query Abfragen für die 14 Stationen der Schweiz

Um die lokalen Temperaturschwankungen darzustellen, haben wir auf der Meteo Schweiz Seite nach geeigneten Daten gesucht. Dabei sind wir auf die homogenen Messdaten von 14 Stationen gestossen. Die Daten werden monatlich sei 1880 aktualisiert. Im Dataset sind Angaben zu Temperaturen sowie Niederschlägsverläufe der letzten 140 Jahren vorhanden.

Die erhobenen Daten der 14 Messstationen sind bereits auf dem Server, die wir nun durch Query-Abfragen abrufen werden.

In [3]:
with SSHTunnelForwarder((ssh_host, ssh_port),ssh_username=ssh_user,ssh_pkey=shh_key,remote_bind_address=(sql_hostname, sql_port)) as tunnel:
        conn = pymysql.connect(host='127.0.0.1', user=sql_username,passwd=sql_password, db=sql_main_database,port=tunnel.local_bind_port)
        cursor = conn.cursor()
        query_BAS = '''SELECT * FROM Suisse_temp_BAS;'''
        query_BER = '''SELECT * FROM Suisse_temp_BER;'''
        query_CHD = '''SELECT * FROM Suisse_temp_CHD;'''
        query_CHM = '''SELECT * FROM Suisse_temp_CHM;'''
        query_DAV = '''SELECT * FROM Suisse_temp_DAV;'''
        query_ENG = '''SELECT * FROM Suisse_temp_ENG;'''
        query_GSB = '''SELECT * FROM Suisse_temp_GSB;'''
        query_GVE = '''SELECT * FROM Suisse_temp_GVE;'''
        query_LUG = '''SELECT * FROM Suisse_temp_LUG;'''
        query_PAY = '''SELECT * FROM Suisse_temp_PAY;'''
        query_SAE = '''SELECT * FROM Suisse_temp_SAE;'''
        query_SIA = '''SELECT * FROM Suisse_temp_SIA;'''
        query_SIO = '''SELECT * FROM Suisse_temp_SIO;'''
        query_SMA = '''SELECT * FROM Suisse_temp_SMA;'''
        query_temp = '''SELECT * FROM Global_temperature;'''
        data_BAS = pd.read_sql_query(query_BAS, conn)
        data_BER = pd.read_sql_query(query_BER, conn)
        data_CHD = pd.read_sql_query(query_CHD, conn)
        data_CHM = pd.read_sql_query(query_CHM, conn)
        data_DAV = pd.read_sql_query(query_DAV, conn)
        data_ENG = pd.read_sql_query(query_ENG, conn)
        data_GSB = pd.read_sql_query(query_GSB, conn)
        data_GVE = pd.read_sql_query(query_GVE, conn)
        data_LUG = pd.read_sql_query(query_LUG, conn)
        data_PAY = pd.read_sql_query(query_PAY, conn)
        data_SAE = pd.read_sql_query(query_SAE, conn)
        data_SIA = pd.read_sql_query(query_SIA, conn)
        data_SIO = pd.read_sql_query(query_SIO, conn)
        data_SMA = pd.read_sql_query(query_SMA, conn)
        data_temp = pd.read_sql_query(query_temp, conn)
        conn.close()
        


#### Umbenennung der Daten
Nun benennen wir die 14 Standorte um, sodass unsere Nutzer/innen die gewünschten Orte auswählen können.Dabei haben wir für eine dreistellige Abkürzung entschieden.

In [4]:
data_BAS['region'] = 'BAS'
data_BER['region'] = 'BER'
data_CHD['region'] = 'CHD'
data_CHM['region'] = 'CHM'
data_DAV['region'] = 'DAV'
data_ENG['region'] = 'ENG'
data_GSB['region'] = 'GSB'
data_GVE['region'] = 'GVE'
data_LUG['region'] = 'LUG'
data_PAY['region'] = 'PAY'
data_SAE['region'] = 'SAE'
data_SIA['region'] = 'SIA'
data_SIO['region'] = 'SIO'
data_SMA['region'] = 'SMA'

schweiz = pd.concat([data_BAS, data_BER,data_CHD,data_CHM,data_DAV,data_ENG,data_GSB,data_GVE,data_LUG,data_PAY,data_SAE,data_SIA,data_SIO,data_SMA])

data_temp['Precipitation'] = '-'
data_temp['region'] = 'Global'

all_temp = pd.concat([schweiz,data_temp])

### Query Abfragen für die CO2 Emissionswerte pro Sektor Global und Schweiz

Um die globalen und lokalen CO2 Emissionswerte zu vergleichen, werden wir im nächsten Abschnitt durch Query-Abfragen die nötigen Tabellen abrufen. 

In [5]:
with SSHTunnelForwarder((ssh_host, ssh_port),ssh_username=ssh_user,ssh_pkey=shh_key,remote_bind_address=(sql_hostname, sql_port)) as tunnel:
        conn = pymysql.connect(host='127.0.0.1', user=sql_username,passwd=sql_password, db=sql_main_database,port=tunnel.local_bind_port)
        cursor = conn.cursor()
        query_suisse_co2_sector_transport = '''SELECT Suisse_CO2_emissions_by_sector.Year, ROUND(Transport / Population,5) AS CO2_emissions_in_tones_per_person FROM Suisse_CO2_emissions_by_sector LEFT JOIN Suisse_population ON Suisse_CO2_emissions_by_sector.Year = Suisse_population.Year'''
        query_suisse_co2_sector_energy = '''SELECT Suisse_CO2_emissions_by_sector.Year, ROUND(Energy / Population,5) AS CO2_emissions_in_tones_per_person FROM Suisse_CO2_emissions_by_sector LEFT JOIN Suisse_population ON Suisse_CO2_emissions_by_sector.Year = Suisse_population.Year;'''
        query_suisse_co2_sector_agriculture = '''SELECT Suisse_CO2_emissions_by_sector.Year, ROUND(Agriculture / Population,5) AS CO2_emissions_in_tones_per_person FROM Suisse_CO2_emissions_by_sector LEFT JOIN Suisse_population ON Suisse_CO2_emissions_by_sector.Year = Suisse_population.Year;'''
        query_suisse_co2_sector_residential = '''SELECT Suisse_CO2_emissions_by_sector.Year, ROUND(Residential / Population,5) AS CO2_emissions_in_tones_per_person FROM Suisse_CO2_emissions_by_sector LEFT JOIN Suisse_population ON Suisse_CO2_emissions_by_sector.Year = Suisse_population.Year;'''
        query_suisse_co2_sector_industry = '''SELECT Suisse_CO2_emissions_by_sector.Year, ROUND(Industry / Population,5) AS CO2_emissions_in_tones_per_person FROM Suisse_CO2_emissions_by_sector LEFT JOIN Suisse_population ON Suisse_CO2_emissions_by_sector.Year = Suisse_population.Year;'''
        query_global_co2_sector_transport = '''SELECT Global_CO2_emissions_by_sector.Year, ROUND(Transport / Population,5) AS CO2_emissions_in_tones_per_person FROM Global_CO2_emissions_by_sector LEFT JOIN Global_population ON Global_CO2_emissions_by_sector.Year = Global_population.Year;'''
        query_global_co2_sector_energy = '''SELECT Global_CO2_emissions_by_sector.Year, ROUND(Energy / Population,5) AS CO2_emissions_in_tones_per_person FROM Global_CO2_emissions_by_sector LEFT JOIN Global_population ON Global_CO2_emissions_by_sector.Year = Global_population.Year;'''
        query_global_co2_sector_agriculture = '''SELECT Global_CO2_emissions_by_sector.Year, ROUND(Agriculture / Population,5) AS CO2_emissions_in_tones_per_person FROM Global_CO2_emissions_by_sector LEFT JOIN Global_population ON Global_CO2_emissions_by_sector.Year = Global_population.Year;'''
        query_global_co2_sector_residential = '''SELECT Global_CO2_emissions_by_sector.Year, ROUND(Residential / Population,5) AS CO2_emissions_in_tones_per_person FROM Global_CO2_emissions_by_sector LEFT JOIN Global_population ON Global_CO2_emissions_by_sector.Year = Global_population.Year;'''
        query_global_co2_sector_industry = '''SELECT Global_CO2_emissions_by_sector.Year, ROUND(Industry / Population,5) AS CO2_emissions_in_tones_per_person FROM Global_CO2_emissions_by_sector LEFT JOIN Global_population ON Global_CO2_emissions_by_sector.Year = Global_population.Year;'''
        data_suisse_co2_sector_transport = pd.read_sql_query(query_suisse_co2_sector_transport, conn)
        data_suisse_co2_sector_energy = pd.read_sql_query(query_suisse_co2_sector_energy, conn)
        data_suisse_co2_sector_agriculturet = pd.read_sql_query(query_suisse_co2_sector_agriculture, conn)
        data_suisse_co2_sector_residential = pd.read_sql_query(query_suisse_co2_sector_residential, conn)
        data_suisse_co2_sector_industry = pd.read_sql_query(query_suisse_co2_sector_industry, conn)
        data_global_co2_sector_transport = pd.read_sql_query(query_global_co2_sector_transport, conn)
        data_global_co2_sector_energy = pd.read_sql_query(query_global_co2_sector_energy, conn)
        data_global_co2_sector_agriculture = pd.read_sql_query(query_global_co2_sector_agriculture, conn)
        data_global_co2_sector_residential = pd.read_sql_query(query_global_co2_sector_residential, conn)
        data_global_co2_sector_industry = pd.read_sql_query(query_global_co2_sector_industry, conn)
        conn.close()

#### Umbenennung der Daten

Für einen besseren Verständnis der Nutzer/innen müssen wir auch hier die Daten umbenennen.

In [6]:
data_suisse_co2_sector_transport['region'] = 'suisse_transport'
data_suisse_co2_sector_energy['region'] = 'suisse_energy'
data_suisse_co2_sector_agriculturet['region'] = 'suisse_agriculture'
data_suisse_co2_sector_residential['region'] = 'suisse_residential'
data_suisse_co2_sector_industry['region'] = 'suisse_industry'
data_global_co2_sector_transport['region'] = 'global_transport'
data_global_co2_sector_energy['region'] = 'global_energy'
data_global_co2_sector_agriculture['region'] = 'global_agriculture'
data_global_co2_sector_residential['region'] = 'global_residential'
data_global_co2_sector_industry['region'] = 'global_industry'


all_CO2 = pd.concat([data_suisse_co2_sector_transport,
                    data_suisse_co2_sector_energy,
                    data_suisse_co2_sector_agriculturet,
                    data_suisse_co2_sector_residential,
                    data_suisse_co2_sector_residential,
                    data_global_co2_sector_transport,
                    data_global_co2_sector_energy,
                    data_global_co2_sector_agriculture,
                    data_global_co2_sector_residential,
                    data_global_co2_sector_industry
                    ])


### Dash Seite erstellen

Wir haben eine lange Zeit mit der Auswahl der Webseitentool verbracht, da für uns die Erstellung einer Webseite über Python nicht bekannt war. Wir haben verschiedene Webseitentools wie bookeh, django, D3 und Vega Lite ausprobiert. Ständig hatten wir grosse Probleme mit dem Python-Backend und aus diesem Grund haben uns für eine Dash-Seite entschieden. Im Gegensatz zu den anderen Tools ist Dash einfach zu bedienen und erfüllt einen grossen Teil unserer Erwartungen. Für Interaktive Visualisierungen ist ein Dashboard ein sehr geeignes Tool.\
Nun erstellen wir eine Dash Seite. 


#### Markdown Text 

In diesem Abschnitt werden wir die Markdown-Texte für die Seite schreiben. 

In [7]:
markdown_text = '''
# ***Global vs Suisse***

## How have global and suisse temperature changed over time?

\n #### A interactive Dashboard is shown below.

'''
markdown_text_2 = '''

## Here you can compare golabal and local CO2 emissions whithin diffrent sectors. 


'''
markdown_text_3 = '''

## About us

\n #### Information about the project
The Federal Office of Meteorology and Climatology (MeteoSwiss) would like to provide the people with better information on climate change. 
A new website is planned, which will present data on climate development in a clear and easy-to-understand way. 
Professionals and interested citizens will be able to obtain an overview of climate change at global and local (localities in Switzerland) level, tailored to their individual needs, with just a few clicks.

\n #### Technical infrastructure
With the help of a Python program, we can download our data, which is in .txt format, from the Meteo Schweiz website. The code can obtain the homogeneous measurement data directly from the site. For the global data we used CSV files. These data were stored on a MySQL relational database. We've created a Ubuntu server on Switch Engine and connected the database and the dash page to it. 

\n #### Database model
After a long research on the web we decided to use a rational database. The decisive argument for a rational database is the experience we already gained last semester. The goal was to deepen our existing knowledge before we got to know other types of databases. The flexibility of the tables and easy linking of the database with Python were among other things also important criteria that spoke in favor of a relational database.

\n #### Team
We are two Data Science students at FHNW Brugg, who have created this site as part of a project.
On the left side you can see Firat Saritas and on right side Kajenthini Kobivasan.
'''

image_filename = 'bild_kayen_firat.jpeg'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


#### Filter Bottons

Für den Filer-Option braucht es Bottons, die es den Nutzer ermöglich gewünschte Daten auszuwählen und miteinader zu vergleichen.

In [8]:
app = dash.Dash(__name__)
df = all_temp

app.layout = html.Div([
    dcc.Markdown(children=markdown_text),
    html.Div([
        html.Br(),
        html.Label(['Period (Choose month)'],style={'font-weight': 'bold', "text-align": "center"}),
        dcc.Dropdown(id='cuisine_month',
            options=[{'label':x, 'value':x} for x in df.sort_values('Month')['Month'].unique()],
            value=1,
            multi=False,
            disabled=False,
            clearable=True,
            searchable=True,
            placeholder='Choose Month',
            className='form-dropdown',
            style={'width':"90%"},
            persistence='string',
            persistence_type='memory')]),
    
    html.Div([
        dcc.Graph(id='our_graph')
    ],className='nine columns'),
    

    dcc.Markdown(children=markdown_text_2),
    html.Div([
        dcc.Graph(id='our_graph_2')
    ],className='nine columns'),

    html.Div([

        html.Br(),
        html.Label(['Choose 2 secors:'],style={'font-weight': 'bold', "text-align": "center"}),
        dcc.Dropdown(id='cuisine_one_2',
            options=[{'label':x, 'value':x} for x in all_CO2.sort_values('region')['region'].unique()],
            value='global_agriculture',
            multi=False,
            disabled=False,
            clearable=True,
            searchable=True,
            placeholder='Choose sector',
            className='form-dropdown',
            style={'width':"90%"},
            persistence='string',
            persistence_type='memory'),

        dcc.Dropdown(id='cuisine_two_2',
            options=[{'label':x, 'value':x} for x in all_CO2.sort_values('region')['region'].unique()],
            value='suisse_agriculture',
            multi=False,
            disabled=False,
            clearable=True,
            searchable=True,
            placeholder='Choose sector',
            persistence='string',
            persistence_type='memory'),


    ],className='two columns'),
    
    dcc.Markdown(children=markdown_text_3),
    html.Div([
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),style={'height': '20%','width': '20%'})])
])

#### Plot Tempertur & CO2 Emission

In [9]:

@app.callback(
    Output('our_graph','figure'),
    [Input('cuisine_month','value')]
)
def build_graph(month):
    dff=df[(df['Month']==month)]
    
    fig = px.line(dff, x="Year", y="Temperature", color='region', height=600)
    fig.update_layout(yaxis={'title':'Temerature'},
                      title={'text':'temperature comparison',
                      'font':{'size':28},'x':0.5,'xanchor':'center'})
    return fig

@app.callback(
    Output('our_graph_2','figure'),
    [Input('cuisine_one_2','value'),
     Input('cuisine_two_2','value')]
)

def build_graph(first_cuisine, second_cuisine):
    dff=all_CO2[(all_CO2['region']==first_cuisine) |
           (all_CO2['region']==second_cuisine)]
    # print(dff[:5])

    fig = px.bar(dff, x="Year", y="CO2_emissions_in_tones_per_person", color='region',height=600)
    fig.update_layout(yaxis={'title':'CO2_emissions_in_tones_per_person'},
                      title={'text':'CO2 comparison',
                      'font':{'size':28},'x':0.5,'xanchor':'center'})
    return fig

#---------------------------------------------------------------

if __name__ == '__main__':
    app.run_server(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Jun/2020 19:40:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Jun/2020 19:40:42] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [05/Jun/2020 19:40:42] "GET /_favicon.ico?v=1.12.0 HTTP/1.1" 200 -
127.0.0.1 - - [05/Jun/2020 19:40:42] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [05/Jun/2020 19:40:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Jun/2020 19:40:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Jun/2020 19:41:44] "POST /_dash-update-component HTTP/1.1" 200 -


### Future Plans
- Niederschlagsdaten einfügen und eine Interaktion ermöglichen
- Die Daten auf einer Weltkarte visualisieren
- Optimierung der Darstellungsinteraktionen 
- Andere CO2-relevante Daten darstellen

Eine Prototyp Seite wurde auf Adobe Xd erstellt, welches im gleichen Order wie dieses Dokument befindet. Dort sind unsere Furte Plans noch visuel ersichtlich. 